In [1]:
import project_utils
from __future__ import annotations
import pandas as pd
import pickle
import project_utils
from sklearn.base import ClassifierMixin
import numpy as np
from typing import Tuple
from pandas import DataFrame
import csv_maker
#pd.set_option('display.max_columns', None)
#pd.set_option('display.expand_frame_repr', False)
#pd.set_option('max_colwidth', None)
import time
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.stats import uniform
np.random.seed(42)



In [2]:
#load the pre-selected features from stepwise
sfs = pickle.load(open('stepwise_selected_features.pickle', 'rb'))
print(sfs.k_feature_names_)


('CO2168.mean', 'CO2336.mean', 'CO242.mean', 'Glob.mean', 'H2O42.mean', 'H2O672.mean', 'H2O84.std', 'NET.std', 'NO168.mean', 'NO168.std', 'NO504.mean', 'NO672.mean', 'NO672.std', 'NO84.mean', 'NOx42.std', 'NOx504.mean', 'PAR.mean', 'RHIRGA168.mean', 'SO2168.mean', 'T168.mean', 'T42.mean', 'T672.mean', 'UV_A.mean')


In [3]:
#setup training data
_x, y, enc = project_utils.Read_data_output_class4()
_x = project_utils.scaling(_x)
x_step = _x[:,sfs.k_feature_idx_]
x_pca, pca_encoder = project_utils.training_with_PCA(_x,15)

In [4]:
#run models
adjusted_weights = {0: 0.24, 1: 0.05, 2: 0.16, 3: 0.55}

c = np.linspace(0,50,100)
l1 = np.linspace(0,1,100)
rf = RandomForestClassifier()
#rf_params = dict(min_samples_split=[2,3,4,5,6], min_samples_leaf=[1,2,3,4,5,6], class_weight=[adjusted_weights])
rf_params = dict(min_samples_split=[2,3,4,5,6], min_samples_leaf=[2,3,4,5,6], class_weight=[adjusted_weights])

lr = LogisticRegression(max_iter=1000)
lr_params = dict( penalty=['l2'], solver=['lbfgs', 'newton-cg', 'sag'], class_weight=[adjusted_weights], multi_class=['multinomial'])

lre = LogisticRegression(max_iter=1000)
lre_params = dict( l1_ratio=l1, penalty=['elasticnet', 'l1', 'l2'], solver=['saga'], class_weight=[adjusted_weights], multi_class=['multinomial'])

nb = GaussianNB(priors=np.array([0.24, 0.05, 0.16, 0.55]))
nb_params = dict()

svm = SVC()
svm_params = dict(kernel=['sigmoid', 'rbf', 'poly'], degree=[2,3], class_weight=[adjusted_weights], probability=[True], decision_function_shape=['ovo', 'ovr'])
#svm_params = dict(C=c,  kernel=['sigmoid', 'rbf', 'poly'], class_weight=[adjusted_weights], probability=[True])


models = [rf,  lre,  nb, svm]
model_params = [rf_params, lre_params, nb_params, svm_params]


model_dict = {}
model_results = None
for m, mp in zip(models, model_params):
    results, md = project_utils.runModelCV(model=m, model_params=mp, x=x_step, y=y, strategy='stepwise', n_iterations=5, k_folds=5)
    if model_results is None:
        model_results = results
    else:
        model_results = pd.concat([model_results, results])
    model_dict.update(md)
for m, mp in zip(models, model_params):
    results, md = project_utils.runModelCV(model=m, model_params=mp, x=x_pca, strategy='pca', y=y, n_iterations=5, k_folds=5)
    if model_results is None:
        model_results = results
    else:
        model_results = pd.concat([model_results, results])
    model_dict.update(md)

In [ ]:
#make testing data from the real set
_test_x, test_y, _enc = project_utils.Read_data_output_class4('npf_test.csv')
_test_x = project_utils.scaling(_test_x)
test_x_step = _test_x[:,sfs.k_feature_idx_]
test_x_pca = pca_encoder.fit_transform(_test_x)

In [ ]:
#do final evaluation
names = []
ids = []
accuracy = []
strat = []
for modelName in model_dict.keys():
    if  model_results.loc[modelName, 'Features'] == 'pca':
        accuracy.append(model_dict[modelName].score(X=test_x_pca, y=test_y))
        strat.append('pca')
    else:
        accuracy.append(model_dict[modelName].score(X=test_x_step, y=test_y))
        strat.append('stepwise')
    na, uid = modelName.split('--')
    names.append(na)
    ids.append(uid)
dftest = pd.DataFrame({'model':names, 'Features': strat, 'accuracy': accuracy}, index=ids)

In [ ]:
dftest.sort_values('accuracy', ascending=False).head(5)

In [ ]:
dftest.groupby(['model', 'Features']).max()

In [ ]:
dftest.groupby(['model', 'Features']).min()

In [ ]:
dftest.groupby(['model', 'Features']).mean()

In [ ]:
dftest.sort_values('accuracy', ascending=False).head()

In [ ]:
#format like this to inspect an individual model
bestModel = model_dict[f'RandomForestClassifier--135540-19']

In [ ]:
bestModel.get_params()